In [1]:
import numpy as np
import tifffile
import pandas
import matplotlib.pyplot as plt

import griottes
import griottes.analyse
import griottes.graphmaker

from griottes.graphmaker import make_spheroids
from griottes.graphmaker import graph_generation_func
from griottes.graphplotter import graph_plot

# 1. Measuring the cell properties

Here we introduce the first function from [Griottes](https://github.com/BaroudLab/Griottes). The role of this function is to analyze the labeled image `labeled_image`. 

You can download a sample image named `sample_image.tiff` from [here](https://drive.google.com/file/d/1XBxLsltppXR-1uRSej1gLtQ_gmXITEd1/view?usp=sharing) and place it into the current folder. Please feel free to contact the repository maintainer if you encounter an issue.

The example image you have downloaded above is a 3D multi-channel image where the 4th channel corresponds to the mask labels designating individual cells. The current library is made to work on 2D and 3D images, with and without the fluorescence channels in the image. The only key criterion is that the channels are ordered in the following order: **Z,X,Y,C**. For example, let's extract the cell positions from a 2D image:

In [ ]:
labeled_image = tifffile.imread('sample_image.tiff')
labeled_image = labeled_image[2,...]

The image is a multi-channel marking for specific biologically relevant markers.

In [ ]:
fig, ax = plt.subplots(1,5, figsize = (10,3))

ax[0].imshow(labeled_image[...,0])
ax[0].axis('off')
ax[0].set_title('DAPI')

ax[1].imshow(labeled_image[...,1])
ax[1].axis('off')
ax[1].set_title('CD45')

ax[2].imshow(labeled_image[...,2])
ax[2].axis('off')
ax[2].set_title('c-Kit')

ax[3].imshow(labeled_image[...,3])
ax[3].axis('off')
ax[3].set_title('Ter 119')

ax[4].imshow(labeled_image[...,4])
ax[4].axis('off')
ax[4].set_title('labels')


The full list of options is available by typing:

```
help(griottes.analyse.cell_property_extraction.get_cell_properties)

```

Here we extract the data from the segmented image only:

In [ ]:
prop_2D = griottes.analyse.cell_property_extraction.get_cell_properties(
    labeled_image,
    mask_channel = 4,
    analyze_fluo_channels = True,
    fluo_channel_analysis_method = 'basic',
    radius = 30,
    min_area = 50,
    ndim = 2
    )

We can also extract more complex pieces of information from the images. Here for example we build a voronoi tesselation separating the volume into different regions for each cell on a 3D image. We also extract the intensities from inside the regions and the geometric properties of the cell nuclei.

In [ ]:
labeled_image = tifffile.imread('sample_image.tiff')

prop_vor, vor = griottes.analyse.cell_property_extraction.get_cell_properties(
    labeled_image,
    mask_channel = 3,
    analyze_fluo_channels = True,
    fluo_channel_analysis_method = 'local_voronoi',
    cell_geometry_properties = True,
    radius = 30,
    labeled_voronoi_tesselation = True,
    percentile = 95,
    min_area = 50
    )

# 2. Building the tissue graph from the data

We now build the graph representing the tissue proper using [Griottes](https://github.com/BaroudLab/Griottes).
 
### Constructing the spheroid graph
 
For example, to build a networkx object strait away from the labeled mask containing the individual cells:

In [ ]:
labeled_image = tifffile.imread('sample_image.tiff')
labeled_image = labeled_image[2,..., 4]

G = graph_generation_func.generate_voronoi_graph(labeled_image,
                                                 dCells = 50)

For more complex outputs, the DataFrame containing the cell information can be passed as an entry. For the purpose of the example, we arbitrarily assign a color to the cells based on the relative intensity levels in a given channel. The `color` descriptor has to be any accepted Matplotlib color.

In [ ]:
descriptors = ['mean_intensity_1', 'color']

color_scheme = ['r', 'g']
prop_2D['color_int'] = (prop_2D.mean_intensity_3 > prop_2D.mean_intensity_3.median()).astype(int)
prop_2D['color'] = [color_scheme[prop_2D.loc[i, 'color_int']] for i in prop_2D.index]

G = graph_generation_func.generate_voronoi_graph(prop_2D, 
                                                 cell_descriptors = descriptors, 
                                                 dCells = 50,
                                                 flat_image = True)

G = graph_generation_func.attribute_layer(G, flat_image = True)


Once the graph has been built, we can plot the spheroid using the plotting functions from the library.

In [ ]:
graph_plot.network_plot_2D(G, 
                           background_image = labeled_image,
                           figsize = (5,5))